+ 1.读取训练集和测试集数据进行预处理
+ 2.训练逻辑回归模型
+ 3.保存模型
+ 4.读取基因数据（基因数据不全，经过预处理大概16000个）
+ 5.定义特征计算函数
+ 6.封装预测函数（输入需要预测的trf序列进行特征计算）
+ 7.调用保存的模型进行预测
+ 8.输出靶标基因并写入csv文件中


In [ ]:
# 加载数据预处理
#upload data and package
import pandas as pd
import mglearn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

trf_train_data = pd.read_csv("E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\train_data.csv")
trf_test_data = pd.read_csv("E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\test_data.csv")



In [ ]:
#设置训练集和测试集数据
train_data = trf_train_data[trf_train_data.columns[9:16]]
train_data = np.array(train_data).tolist()

test_data = trf_test_data[trf_test_data.columns[9:16]]
test_data = np.array(test_data).tolist()

In [ ]:
#设置标签数据
train_data_label = np.array(trf_train_data).tolist()
train_data_label = [i[16] for i in train_data_label]

test_data_label = np.array(trf_test_data).tolist()
test_data_label = [i[16] for i in test_data_label]

In [ ]:
X_train = train_data
X_test = test_data
y_train = train_data_label
y_test = test_data_label

In [ ]:
#使用逻辑回归模型(0.1)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5,shuffle=True, random_state=0)
#正则化系数C = 1时(可调节参数C)
logreg = LogisticRegression(C = 0.001).fit(X_train, y_train)


print("Training set score: {:.3f}".format(logreg.score(X_train, y_train)))
print("Test set score: {:.3f}".format(logreg.score(X_test, y_test)))
print("The feature weight: {}".format(logreg.coef_))

In [ ]:
#保存模型
import joblib
joblib.dump(logreg,"E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\logreg.pkl")

In [ ]:
#读取基因列表
gene_list = pd.read_csv("E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\unique_gene.csv")
gene_list = gene_list.iloc[:,1:5]

In [ ]:
#定义特征计算函数

#1.创建一个trf序列的列表
def create(trf):
    seq = [trf]
    trf_seq = pd.DataFrame(seq*16606)
    trf_seq.columns = ["trf_seq"]
    return trf_seq

#计算AU含量
def AU_contant(trf_feature):
    AU_list = pd.DataFrame([0]*len(trf_feature))
    AU_list.columns = ["AU_contant"]
    for i in range(len(trf_feature)):
        upscore = 0
        downscore = 0
        for j in range(30):
            if(trf_feature.iloc[i,2][j] == "A" or trf_feature.iloc[i,2][j] == "T"):
                upscore += 1/(j+1)
            if(trf_feature.iloc[i,3][j] == "A" or trf_feature.iloc[i,3][j] == "T"):
                downscore += 1/(j+1)
        AU_list.iloc[i,0] = (upscore + downscore)/2
    trf_feature = pd.concat([trf_feature,AU_list],axis = 1)
    return trf_feature

#看该位点是否匹配
def ispaired(nt):
    if nt == "A":
        return "T"
    if nt == "T":
        return "A"
    if nt == "G":
        return "C"
    if nt == "C":
        return "G"
    else:
        return ""

# 确定结合区位置
def bind_loc(trf_feature):
    start_loc = pd.DataFrame([0]*len(trf_feature))
    start_loc.columns = ["start_loc"]
    end_loc = pd.DataFrame([0]*len(trf_feature))
    end_loc.columns = ["end_loc"]
    for i in range(len(trf_feature)):
        seq = trf_feature.iloc[i,0][0]
        for j in range(len(trf_feature.iloc[i,2])):
            if ispaired(seq) == trf_feature.iloc[i,2][j]:
                start_loc.iloc[i,0] = j
                end_loc.iloc[i,0] = j + len(trf_feature.iloc[i,0])
                break
    trf_feature = pd.concat([trf_feature,start_loc],axis = 1)
    trf_feature = pd.concat([trf_feature,end_loc],axis = 1)
    return trf_feature


#统计结合位点数    
def bind_num(trf_feature):
    count_paired = pd.DataFrame([0]*len(trf_feature))
    count_paired.columns = ["count_paired"]
    for i in range(len(trf_feature)):
        count_num = 0
        for j in range(min(len(trf_feature.iloc[i,0]),len(trf_feature.iloc[i,2])-trf_feature.iloc[i,6])):
            if ispaired(trf_feature.iloc[i,0][j]) == trf_feature.iloc[i,2][trf_feature.iloc[i,6]+j]:
                count_num += 1
        count_paired.iloc[i,0] = count_num
    trf_feature = pd.concat([trf_feature,count_paired],axis = 1)
    return trf_feature

#计算字符串中最长的连续字符数
def maxDup(strs):
    n=len(strs)
    maxlen=1
    length=1
    maxstr=[]
    for i in range(n-1): 
        if strs[i]==strs[i+1]:
            length+=1
            maxstr.append(strs[i+1])
            if length>maxlen:
                maxlen=length
        else:
            length=1
    return maxlen


#计算结合区的长度
def bind_length(trf_feature):
    max_length = pd.DataFrame([0]*len(trf_feature))
    max_length.columns = ["max_length"]
    pos_longest = pd.DataFrame([0]*len(trf_feature))
    pos_longest.columns = ["pos_longest"]
    for i in range(len(trf_feature)):
        seq = ""
        for j in range(min(len(trf_feature.iloc[i,0]),len(trf_feature.iloc[i,2])-trf_feature.iloc[i,6])):
            if ispaired(trf_feature.iloc[i,0][j]) == trf_feature.iloc[i,2][trf_feature.iloc[i,6]+j]:
                seq += "T"
            else:
                seq += str(j)
        max_length.iloc[i,0] = maxDup(seq)
        pos_longest.iloc[i,0] = len(seq)
    trf_feature = pd.concat([trf_feature,max_length],axis = 1)
    trf_feature = pd.concat([trf_feature,pos_longest],axis = 1)
    return trf_feature
        
#计算3'区的结合位点数
def three_prime(trf_feature):
    three_prime = pd.DataFrame([0]*len(trf_feature))
    three_prime.columns = ["three_prime"]
    for i in range(len(trf_feature)):
        count_num = 0
        for j in range(min(7,len(trf_feature.iloc[i,2])-trf_feature.iloc[i,6])):
            if ispaired(trf_feature.iloc[i,0][j]) == trf_feature.iloc[i,2][trf_feature.iloc[i,6]+j]:
                count_num += 1
        three_prime.iloc[i,0] = count_num
    trf_feature = pd.concat([trf_feature,three_prime],axis = 1)
    return trf_feature
    
#输出trf特征列表   
def feature_calculation(trf_seq,trf_list):
    trf = create(trf_seq)
    trf_feature = pd.concat([trf,trf_list],axis = 1)
    trf_feature = AU_contant(trf_feature)
    trf_feature = bind_loc(trf_feature)
    trf_feature = bind_num(trf_feature)
    trf_feature = bind_length(trf_feature)
    trf_feature = three_prime(trf_feature)
    return trf_feature

In [ ]:
#计算特征
trf_sequence = input("please input your trf sequence:")
trf_feature = feature_calculation(trf_sequence, gene_list)

In [ ]:
#获取测试集数据
test_data = trf_feature.iloc[:,5:12]
test_data = np.array(test_data).tolist()

In [ ]:
#调用模型
import joblib
logreg = joblib.load("E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\logreg.pkl")
#输入测试集数据进行预测，并输出靶标基因名
result_label = logreg.predict(test_data)
target_gene = []
for i in range(len(result_label)):
    if result_label[i] == 1:
        print(trf_feature.iloc[i,1])
        target_gene.append(trf_feature.iloc[i,1])

In [ ]:
#将靶标基因写入csv文件中
target = pd.DataFrame(columns = ["gene"],data = target_gene)
target.to_csv("E:\\Windows-SSD\\Program Files (x86)\\Common Files\\Designer\\tRF_prediction\\data\\target_gene.csv")